# Data -1 | Link

## Link KNN with Turnout

A list of elections in North Carolina https://en.wikipedia.org/wiki/Elections_in_North_Carolina.

In [ ]:
from datalink import *

with open(path_0 + 'turnout_history_20190101.pkl','rb') as f: 
    history = pickle.load(f)
elections = history.election_desc.unique()

# 'MUNCIPAL', 'MUNI'
# 'EDUCATION'
# 'ALCHOHOL', 'BEVERAGE'

election_types = ['GENERAL', 'PRIMARY', 'MUNICIPAL',]# 'SCHOOL', 'REFERENDUM', 'MUNI AND SANI DISTRICT', 'ABC']
election_types = {election_type: [e for e in elections if election_type in e] for election_type in election_types}


files = set([x.split('_chunk_')[0] for x in os.listdir(path_1) if x.split('.')[-1] == 'pkl'])
for file in sorted(files):
    
    state, year = file.split('_')
    print(file)
    
    last_election_type = [e for e in election_types][-1]
    chunk_file_names = [x for x in os.listdir(path_1) if file in x]
    chunk_paths = [x.strip(file) for x in chunk_file_names]
    max_chunk = max([x for x in np.arange(0,30) if str(x) in '_'.join(chunk_paths)])
    
    if file + '_' + last_election_type + '_chunk_' + str(max_chunk) + '.pkl' not in os.listdir(path_link):
        
        knn_voters = pd.read_pickle(path_4+'knn_' + year + '_' + str(k_max) + 'k.pkl')
        
        # this is crashing the kernal!^
        # I need to rewrite knn for chunks
        
        knn_idu_list = knn_voters.idu.unique()
        print('  KNN:',len(knn_voters))

        for election_type in election_types:
            if year + '_' + election_type + '.pkl' not in os.listdir(path_link):
                print('  ',election_type)
                keep_election_cols = ['voted_' + e for e in election_types[election_type] if (year in e) or (str(int(year) - 1) in e)]
                
                for chunk_name in sorted(chunk_file_names):
                    knn_chunk_name = chunk_name.replace('chunk','knn_' + election_type + '_chunk')

                    chunk = pd.read_pickle(path_1 + chunk_name)
                    chunk = chunk[['idu', 'D', 'R', 'O'] + keep_election_cols]
                    chunk = knn_voters.merge(chunk, on='idu', how='inner')
                    
                    chunk.to_pickle(path_link + 'chunks/' + knn_chunk_name)
                    print('  ||',len(chunk),'voters')
                    
            else:
                print('  ||',election_type,'Done')
        del knn_voters
        
    else:
        print('  || Done')

NC_2010


## ACS and Squares

Add ACS interpolated data to knn samples


In [ ]:
state = 'NC'
linkpath = '../voters/data5_link/'+state+'/'

with open('../voters/data4_knn/'+state+'/key_id_subsamples.pkl','rb') as f: key_id_subsamples = pickle.load(f)
keys = [key for key in key_id_subsamples]
datadict = {}
link_ids = []
for key in keys:
    if key+'_plus.csv' in os.listdir(linkpath):
        datadict[key] = pd.read_csv(linkpath+key+'_plus.csv', low_memory=False)
    else:
        datadict[key] = pd.read_csv(linkpath+key+'.csv', low_memory=False)
    link_ids = link_ids + list(datadict[key].DOUBLE_ID.unique())
link_ids = list(set(link_ids)) # List of subsample DOUBLE_IDS

meterlist = [500]#,2000]
for meters in meterlist:
    datalist = []
    for year in nc_yearlist:
        with open('../voters/data3_neighborhoods/'+state+'/SNd_'+str(year)+'_'+str(meters)+'m.pkl','rb') as f: SNd = pickle.load(f)
        finished = []
        for ni in SNd:
            n = SNd[ni]
            n['YEAR'] = year
            n['M_'+str(meters)+'m'] = sum((n.GENDER == 'M')*1)
            n['F_'+str(meters)+'m'] = sum((n.GENDER == 'F')*1)
            n['BLACK_'+str(meters)+'m'] = sum((n.RACE == 'BLACK or AFRICAN AMERICAN')*1)
            n['WHITE_'+str(meters)+'m'] = sum((n.RACE == 'WHITE')*1)
            n['OTHER_'+str(meters)+'m'] = sum((~n.RACE.isin(['BLACK or AFRICAN AMERICAN','WHITE']))*1)
            n['D_'+str(meters)+'m'] = n.D.sum()
            n['R_'+str(meters)+'m'] = n.R.sum()
            n['O_'+str(meters)+'m'] = n.O.sum()
            n['MEAN_AGE_'+str(meters)+'m'] = n.AGE.astype(float).mean()
            print(year,round(len(finished)/len(SNd)*100,2),'%')
            clear_output(wait=True)
            finished.append(ni)
        SNfinal = [SNd[i] for i in SNd]
        SNfinal = pd.concat(SNfinal,ignore_index=True, sort=False)
        SNfinal['DOUBLE_ID'] = SNfinal['ID_NUM'] + SNfinal['ID_NUM_a']
        SNfinal = SNfinal[SNfinal.DOUBLE_ID.isin(link_ids)]
        datalist.append(SNfinal)

    linkdata = pd.concat(datalist)
    linkdata = linkdata.sort_values(by=['DOUBLE_ID','YEAR'])
    
    keys = ['move','switch','stay','move_switch']
    for key in keys:
        key_linkdata = linkdata[linkdata.DOUBLE_ID.isin(datadict[key].DOUBLE_ID.unique())]
        key_linkdata['DOUBLE_ID_YEAR'] = key_linkdata['DOUBLE_ID'] + '_' + key_linkdata['YEAR'].astype(str)
        key_linkdata_droplist = ['ID_NUM','CITY','STATE','ZIP_CODE','ID_NUM_a','GENDER','AGE','RACE','sn_i','lat','lon','YEAR','D','R','O']
        key_linkdata_keeplist = [col for col in key_linkdata.columns.values if col not in key_linkdata_droplist]
        key_linkdata = key_linkdata[key_linkdata_keeplist]

        datadict[key]['DOUBLE_ID_YEAR'] = datadict[key]['DOUBLE_ID'] + '_' + datadict[key]['YEAR'].astype(str)

        merge = datadict[key].merge(key_linkdata, left_on='DOUBLE_ID_YEAR', right_on='DOUBLE_ID_YEAR', how='outer')
        rename_dict = {x:x.replace('_x','') for x in squares_keep_cols if '_x' in x}
        merge = merge.rename(columns=rename_dict)

        merge.to_csv('data5_link/NC/'+key+'_plus.csv')

In [ ]:
merge.columns.values

In [66]:
#autor_variables = ['POP', 'WHITE', 'BLACK', 'VOTING_POP', 'LABOR',
#       'COLLEGE', 'COLLEGE_EMPLOYED', 'SOME_COLLEGE',
#       'SOME_COLLEGE_EMPLOYED', 'NO_COLLEGE', 'NO_COLLEGE_EMPLOYED',
#       'TRANTIME', 'COLLEGE_WAGE', 'SOME_COLLEGE_WAGE', 'NO_COLLEGE_WAGE']
#for col in autor_variables:
#    move[col+'_past'] = move[col].shift(1)
#move = move[move['DOUBLE_ID'] == move['DOUBLE_ID'].shift(1)]
#move.to_csv('data5_link/NC/'+keys[0]+'_plus.csv')

In [39]:
#for col in ['DOUBLE_ID','ZIP_CODE','sn_i','lat','lon','D','R','O']:#+return_k_col_names(state):
#    colshiftname = col+'_past'
#    linkdata[colshiftname] = linkdata[col].shift(1)
#linkdata = linkdata[linkdata['DOUBLE_ID'] == linkdata['DOUBLE_ID'].shift(1)]

In [30]:
from datetime import date, time, datetime
#date.today()
datetime.now().strftime('%H')

'12'